In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import scipy
import glob
import os

In [8]:
CIs = []

def import_dfs():
    # Path to the folder containing CSV files
    folder_path = '../dados/'

    # Get a list of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    # Initialize an empty list to hold dataframes
    data_frames = []

    # Read the first CSV file to establish the schema
    schema = ['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm','apa_t']


    # Load each remaining CSV file, reorder columns, and append to the list
    for file in csv_files:
        df = pd.read_csv(file)  
        print(file)
        df = df[schema]  # Reorder columns to match the schema
        data_frames.append(df)
        CIs.append(df.iloc[0,:4].to_numpy())
    csv_files = [name.split('\\')[-1].rstrip('.csv') for name in csv_files]
    return data_frames,csv_files 
data_frames,file_list = import_dfs()
print(data_frames[0].columns)
print(file_list)
print(CIs)

../dados\25.45nh60ab20.csv
../dados\25.46nh30ab30.csv
../dados\25.47nh5ab80.csv
../dados\5.100nh40ab80.csv
../dados\5.102nh20ab40.csv
../dados\5.103nh40ab30.csv
../dados\5.106nh100ab40.csv
../dados\5.107nh55ab55.csv
../dados\5.109nh5ab55.csv
../dados\5.110nh10ab55.csv
../dados\5.112nh78ab35.csv
../dados\5.48nh20ab80.csv
../dados\5.80nh12ab40.csv
../dados\5.81nh5ab30.csv
../dados\nh60ab80.csv
Index(['ester_mm', 'amox_mm', 'apa_mm', 'aoh_mm', 'apa_t'], dtype='object')
['25.45nh60ab20', '25.46nh30ab30', '25.47nh5ab80', '5.100nh40ab80', '5.102nh20ab40', '5.103nh40ab30', '5.106nh100ab40', '5.107nh55ab55', '5.109nh5ab55', '5.110nh10ab55', '5.112nh78ab35', '5.48nh20ab80', '5.80nh12ab40', '5.81nh5ab30', 'nh60ab80']
[array([20.,  0., 60.,  0.]), array([30.,  0., 30.,  0.]), array([80.,  0.,  5.,  0.]), array([80.,  0., 40.,  0.]), array([40.5       ,  0.        , 21.75      ,  3.37262013]), array([30.4 ,  0.  , 43.  ,  2.55]), array([4.00389864e+01, 6.40149834e-02, 1.00045181e+02, 1.42108547e+0

In [12]:
# luci parameters

kcat1        = 0.178 #Constante catal�tica do consumo do �ster (�mol/i.u. per min)
 
kcat2        = 0.327 #Constante catal�tica da hidr�lise da amoxicilina (�mol/i.u. per min)
 
Km1          = 7.905 #Constante de Michaelis-Menten ou constante de afinidade para consumo do �ster(mM) 
 
Km2          = 12.509 #Constante de Michaelis-Menten ou constante de afinidade para hidr�lise da amoxicilina(mM)
 
Tmax         = 0.606 #Taxa de convers�o m�xima do complexo acil-enzima-n�cleo em produto
 
Ken          = 14.350 #Constante de adsor��o do 6-APA
 
kAB          = 3.78 #Constante de inibi��o do �ster(POHPGME)(mM)
 
kAN          = 9.174 #Constante de inibi��o da amoxicilina (mM)
 
kAOH         = 10.907 #Constante de inibi��o do POHPG, produto da hidr�lise da amoxicilina (mM)
 
kNH          = 62.044 #Constante de inibi��o do 6-APA

luci_P = np.zeros(10)
luci_P[0]   = kcat1    
luci_P[1]   = kcat2    
luci_P[2]   = Km1      
luci_P[3]   = Km2      
luci_P[4]   = Tmax     
luci_P[5]   = Ken      
luci_P[6]   = kAB      
luci_P[7]   = kAN      
luci_P[8]   = kAOH     
luci_P[9]  = kNH 

In [9]:
def enzymic_amox(t,y, 
kcat1,
kcat2,
Km1,
Km2,  
Tmax, 
Ken,  
kAB,  
kAN,  
kAOH, 
kNH):
    FAB = 0
    FNH = 0 
    
    CAB = y[0]
    CAN = y[1]
    CNH = y[2]
    CAOH = y[3]

    Cez = 1

    # Consumo de ester
    VAB = (kcat1*CAB*Cez)/((Km1*(1 + (CAN/kAN) + (CAOH/kAOH))) + CAB)
    
    # Hidrolise de amoxicilina
    VAN = (kcat2*CAN*Cez)/((Km2*(1 + (CAB/kAB) + (CNH/kNH) + (CAOH/kAOH))) + CAN)
    
    # Enzima saturada com 6-apa
    X   = CNH/(Ken + CNH)
    
    # Sintese enzimatica
    VS  = VAB*Tmax*X

    # Hidrolise de ester
    Vh1 = (VAB - VS) 

    dy = np.zeros(4)

    # C. ester
    dy[0] = ((-(VS - VAN) - (Vh1 + VAN)) + FAB) 
    
    # C. amox
    dy[1] = (VS - VAN)                         
    
    # C. 6-apa
    dy[2] = (-(VS - VAN) + FNH)                
    
    # C. POHPG
    dy[3] =  (Vh1 + VAN)
    
    return np.array(dy)  

In [10]:
def ode15s_amox(P, CI, t):
    return scipy.integrate.solve_ivp(enzymic_amox,t_span=(t[0],t[-1]),t_eval=t,y0=CI,method='BDF',args=P).y.T

In [83]:
def mcmc(P,name,priori,N=5000,ode_solver=ode15s_amox,status=False):

    Np = len(P)
    # Extraindo dados
    exps = []
    times = []
    
    for i in range(len(CIs)):
        t = data_frames[i].loc[:,'apa_t'].dropna().to_numpy()

        exp = np.zeros((len(t),4))
        
        exp[:,0]  = data_frames[i].loc[:,'ester_mm'].dropna().to_numpy()
        exp[:,1]  = data_frames[i].loc[:,'amox_mm'].dropna().to_numpy()
        exp[:,2] = data_frames[i].loc[:,'apa_mm'].dropna().to_numpy()
        exp[:,3]  = data_frames[i].loc[:,'aoh_mm'].dropna().to_numpy()
        exps.append(exp)

        times.append(t)
    print(len(exps))
    Cez = 1
    refs = []
    desvio = np.zeros((4,len(CIs)))

    for i in range(len(CIs)):
        t = times[i]
        ref = np.zeros((len(t),4))
        Y = ode_solver(P,CIs[i],t)
        ref[:,0]   =  Y[:,0]      
        ref[:,1]   =  Y[:,1]    
        ref[:,2]   =  Y[:,2]      
        ref[:,3]  =   Y[:,3]

        refs.append(ref)

        desvio[0,i] =  0.1*max(ref[:,0]) 
        desvio[1,i] =  0.1*max(ref[:,1])
        desvio[2,i] =  0.1*max(ref[:,2])
        desvio[3,i] =  0.1*max(ref[:,3])
    
    waux      = 6e-3 
    media_g   = 1       #Média gaussiana
    desviop_g = 0.6     #Desvio dos parâmetros ao utilizar priori gaussian

    estimate = [i for i in range(10)]
    Nfix = len(estimate)

    p_ref = P                   
    p_old = P                     

    media_MCMC = media_g*P
    desvio_P   = desviop_g*P

    w = np.ones((1,Np))   
    w = w*waux  

    parametro_exato = (p_ref*np.ones((N,1))).T
    
    aceitacao    = np.zeros((1,N))  
    cadeia       = np.zeros((Np,N)) 
    conv_likeli  = np.zeros((1,N))
    k=0
    lk_old = 0
        
    for i in range(len(CIs)):
        Lk_1 = np.dot((exps[i][:,0] - refs[i][:,0]),(exps[i][:,0] - refs[i][:,0]).T) / (desvio[0,i]**2)     
        Lk_2 = np.dot((exps[i][:,1] - refs[i][:,1]),(exps[i][:,1] - refs[i][:,1]).T) / (desvio[1,i]**2)      
        Lk_3 = np.dot((exps[i][:,2] - refs[i][:,2]),(exps[i][:,2] - refs[i][:,2]).T) / (desvio[2,i]**2)       
        Lk_4 = np.dot((exps[i][:,3] - refs[i][:,3]),(exps[i][:,3] - refs[i][:,3]).T) / (desvio[3,i]**2)  
        lk_partial = Lk_1 + Lk_2 + Lk_3 + Lk_4
        lk_old += lk_partial
    

    prior_old = np.sum((((p_old - media_MCMC))/((desvio_P)**2)))
    cadeia[:,0] = p_old
    
    # Contador para verificar a aceitação
    for i in range(0, N):  # Python index starts at 0, MATLAB at 1
        progress = (i / N) * 100
        if status:
            print(progress,end='\r')
        # New parameter vector
        P_new = p_old + w * np.random.randn(Np) * p_old
        #print('p_new: ', P_new)
        # Simulate using ODE solver

        chain_ref = []
        for j in range(len(CIs)):
            t = times[j]
            ref = np.zeros((len(t),4))

            Y = ode_solver(P_new[0],CIs[j],t)
            ref[:,0]  =  Y[:,0]      
            ref[:,1]  =  Y[:,1]    
            ref[:,2]  =  Y[:,2]      
            ref[:,3] =   Y[:,3]
            chain_ref.append(ref)


        # Calculate new prior
        Prior_new = np.sum(((P_new[0] - media_MCMC) / desvio_P) ** 2)

        Lk_new = 0
        # Likelihood calculations
        for j in range(len(CIs)):
            Lk_1 = np.dot((exps[j][:,0] - chain_ref[j][:,0]),(exps[j][:,0] - chain_ref[j][:,0]).T) / (desvio[0,j]**2)     
            Lk_2 = np.dot((exps[j][:,1] - chain_ref[j][:,1]),(exps[j][:,1] - chain_ref[j][:,1]).T) / (desvio[1,j]**2)      
            Lk_3 = np.dot((exps[j][:,2] - chain_ref[j][:,2]),(exps[j][:,2] - chain_ref[j][:,2]).T) / (desvio[2,j]**2)       
            Lk_4 = np.dot((exps[j][:,3] - chain_ref[j][:,3]),(exps[j][:,3] - chain_ref[j][:,3]).T) / (desvio[3,j]**2)  
            lk_partial = Lk_1 + Lk_2 + Lk_3 + Lk_4
            Lk_new += lk_partial 
    
        # MCMC acceptance check
        if np.log(random.random()) < (-0.5 * (Lk_new + Prior_new - lk_old - prior_old)):
            p_old = P_new
            lk_old = Lk_new
            prior_old = Prior_new
            k += 1
        
        # Store results
        aceitacao[0,i] = k
        cadeia[:, i] = p_old
        conv_likeli[0,i] = lk_old + prior_old
    print(P_new)
    fig,ax = plt.subplots(2,5,figsize=[20,8])
    ax = ax.flatten()
    labels = ['kcat1',
    'kcat2',
    'Km1',
    'Km2',  
    'Tmax', 
    'Ken',  
    'kAB',  
    'kAN',  
    'kAOH', 
    'kNH']
    for idx,g in enumerate(ax):
        g.plot(cadeia[idx,:])    
        g.set_title(labels[idx])
        g.set_ylabel("Value")
        g.set_xlabel("Iteration")
    fig.savefig(f"./chains/{priori}_{N}_{name}.png",dpi=400)
    plt.close()    

    aq = int(0.9 * N)  # Starting index for the burn-in period
    IC = 0.99          # Confidence interval level

    # Confidence interval bounds
    xaux = (1 - IC) / 2
    IC_inf = xaux
    IC_sup = 1 - xaux
    estimate = [i for i in range(Np)]

    amostra = N - aq + 1
    amostras = [] # [CIs, Nics, t, subs]


    output_par = np.zeros((amostra,Np))
    for i in range(len(CIs)):
        t = times[i]
        inst_amostra = np.zeros((len(list(range(aq, N + 1))),len(t),4))

        for idx,j in enumerate(range(aq, N + 1)):
            ii = j - aq
            paux = cadeia[:, j - 1]
            output_par[ii,:] = paux
            
            Y = ode_solver(paux,CIs[i],t)
            inst_amostra[idx,:,0]  = Y[:,0]
            inst_amostra[idx,:,1] = Y[:,1]
            inst_amostra[idx,:,2] = Y[:,2]
            inst_amostra[idx,:,3] = Y[:,3]
        
        amostras.append(inst_amostra)


    AB_media = []
    AN_media = []
    NH_media = []
    AOH_media =[]
    AB_sup = []
    AN_sup = []
    NH_sup = []
    AOH_sup =[]
    AB_inf = []
    AN_inf = []
    NH_inf = []
    AOH_inf =  []   

    for i in range(len(CIs)):
        t = times[i]
        amostra = amostras[i]
        
        AB_mean = np.zeros(len(t))
        AB_inferior = np.zeros(len(t))
        AB_superior = np.zeros(len(t))
        NH_mean = np.zeros(len(t))
        NH_inferior = np.zeros(len(t))
        NH_superior = np.zeros(len(t))

        for j in range(len(t)):    
            AB_mean[j] = np.mean(amostra[:,j,0])
            y = np.percentile(amostra[:, j,0], [IC_inf * 100, IC_sup * 100])
            AB_inferior[j] = y[0]
            AB_superior[j] = y[1]

            NH_mean[j] = np.mean(amostra[:, j,2])
            y = np.percentile(amostra[:, j,2], [IC_inf * 100, IC_sup * 100])
            NH_inferior[j] = y[0]
            NH_superior[j] = y[1]
        
        AN_mean = np.zeros(len(t))
        AN_inferior = np.zeros(len(t))
        AN_superior = np.zeros(len(t))
        AOH_mean = np.zeros(len(t))
        AOH_inferior = np.zeros(len(t))
        AOH_superior = np.zeros(len(t))
        
        for j in range(len(t)):
            AN_mean[j] = np.mean(amostra[:, j, 1])
            y = np.percentile(amostra[:, j,1], [IC_inf * 100, IC_sup * 100])
            AN_inferior[j] = y[0]
            AN_superior[j] = y[1]

            AOH_mean[j] = np.mean(amostra[:, j,3])
            y = np.percentile(amostra[:, j,3], [IC_inf * 100, IC_sup * 100])
            AOH_inferior[j] = y[0]
            AOH_superior[j] = y[1]

        AB_media.append(AB_mean)
        NH_media.append(NH_mean)
        AB_inf.append(AB_inferior)
        AB_sup.append(AB_superior)
        NH_inf.append(NH_inferior)
        NH_sup.append(NH_superior)
        AN_media.append(AN_mean)
        AOH_media.append(AOH_mean)
        AN_inf.append(AN_inferior)
        AN_sup.append(AN_superior)
        AOH_inf.append(AOH_inferior)
        AOH_sup.append(AOH_superior)

    # individual plot
    # Ensure 'results' directory exists
    results_dir = 'results_mcmc/'

    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    for i in range(len(CIs)):
        names = ['POH-PGME','Amoxicillin','6-APA','POHPG']
        t = times[i]
        style = ['or','og','ob','ok']
        mstyle = ['-r','-g','-b','-k']
        medidas = [
            [AB_media[i], AB_inf[i], AB_sup[i]],
            [AN_media[i], AN_inf[i], AN_sup[i]],
            [NH_media[i], NH_inf[i], NH_sup[i]],
            [AOH_media[i],AOH_inf[i],AOH_sup[i]]
        ]
        fig,ax = plt.subplots(2,2,figsize=(10,10))
        for idx,axis in enumerate(ax.flatten()):
            sname = names[idx]
            medida = medidas[idx]
            
            axis.plot(t, exps[i][:,idx], style[idx], label=sname)
            axis.plot(t, medida[0], mstyle[idx])
            axis.plot(t, medida[1], '--r',linewidth=0.5)
            axis.plot(t, medida[2], '--r',linewidth=0.5) 
            axis.set_xlabel('Tempo (min)')
            axis.set_ylabel(f'Concentração de {sname} (mm)')  # Use f-string for proper string formatting
            axis.legend()  # Adjust legend position
        fig.savefig(os.path.join(results_dir, f"{i}_{priori}_{N}_{name}_mcmc_multi.png"), dpi=300)
        plt.close()  # Close the figure to release memory
    return output_par
        


In [86]:
mcmc(P=luci_P,name='test',priori='luci',N=1000,status=True)

15
[[ 0.37124743  0.50065318 29.8235526  11.23199683  0.84617183 21.21718171
   4.04183358  6.0424692   7.03132378 46.35599691]]


array([[ 0.33915686,  0.48942752, 26.83524022, ...,  6.32213622,
         7.03535138, 47.14349819],
       [ 0.34385999,  0.48771199, 26.99654438, ...,  6.38204239,
         7.09183163, 46.81967431],
       [ 0.34385999,  0.48771199, 26.99654438, ...,  6.38204239,
         7.09183163, 46.81967431],
       ...,
       [ 0.377709  ,  0.50031097, 30.10759733, ...,  6.06183792,
         7.01494302, 46.46572431],
       [ 0.37320137,  0.49888898, 29.98574541, ...,  6.06450425,
         7.01584228, 46.67572545],
       [ 0.37124743,  0.50065318, 29.8235526 , ...,  6.0424692 ,
         7.03132378, 46.35599691]])